### 휴먼지능정보공학과 201910787 김영서

## 연습문제

주의:

 - 다른 과제보다 많이 약 10% 배점합니다

 - 자신의 힘으로 풀어야 합니다. 코드 유사성 없어야 합니다.



A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.



함수 | 설명
-----|-----
setA | 무작위로 가위, 바위, 보를 생성하고, 설정한다. ```1000 wei```씩 금액을 건다.
setB | 외부에서 직접 입력하여 설정한다. ```1000 wei```씩 금액을 건다.
play | 가위, 바위, 보의 승패를 정한다.
distributeBetAmount | 이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 ```1000 wei```를 가지게 된다.
getMatchResult() | "A wins", "B wins", "tie" 결과를 출력



ganache@8345에서 게임을 다음과 같이 진행하세요.

- 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력

- A가 내기금액 걸고, 컴퓨터에서 무작위로 선택한 가위바위보

- B가 내기금액 걸고, 사용자가 입력한 가위바위보

- 승부내기

- 누가 이겼는지 출력

- 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력


geth@8445에서 동일하게 실행하세요 (배점 30%)

In [55]:
%%writefile src/Rsp.sol
pragma solidity ^0.5.10;
contract Rsp {
    struct Player {
        string name;
        uint batting;
        uint setRsp;
    }
    int256 result = -1;
    address payable owner;
    address addA;
    address addB;
    uint balance;
    constructor() public {
        owner = msg.sender;
    }
    mapping(address => Player) playerMap;
    mapping(string => address) addressByName;
    mapping(address => uint) balanceOf;
    
    function setAddressAB(address receiver)public {
        addA = address(this);
        addB = receiver;
    }
    function deposit(uint amount) payable public onlyOwner { 
        require(msg.value == amount);
        balanceOf[addA]+=amount;
        balanceOf[addB]+=amount;
    }
    function setA() public {
        uint _rsp = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        Player memory m=Player("Player_A",1000,_rsp);
        playerMap[addA]=m;
        addressByName["Player_A"]=addA;
    }
    function setB(uint _batting, uint _rsp) public {
        Player memory m2=Player("Player_B",_batting,_rsp);
        playerMap[addB]=m2;
        addressByName["Player_B"]=addB;
    }
    function play() public {
        // 0:rock 1:scissor : 2:paper
        string memory name1="Player_A";
        string memory name2="Player_B";
        Player memory u1=playerMap[addressByName[name1]];
        Player memory u2=playerMap[addressByName[name2]];
        uint a=u1.setRsp;
        uint b=u2.setRsp;
        if(a==b) {
            result=0; // tie
        }
        else if((a==0&&b==1)||(a==1&&b==2)||(a==2&&b==0)) {
            result=1; // a wins
        }
        else {
            result=2; // b wins
        }
    }
    function distributeBetAmount() public payable {
        string memory name1="Player_A";
        string memory name2="Player_B";
        Player memory u1=playerMap[addressByName[name1]];
        Player memory u2=playerMap[addressByName[name2]];
        uint a_batting=u1.batting;
        uint b_batting=u2.batting;
        if(result==1) {
            balanceOf[addressByName[name2]]-=b_batting;
            balanceOf[addressByName[name1]]+=b_batting;
        }
        else if(result==2) {
            balanceOf[addressByName[name2]]+=a_batting;
            balanceOf[addressByName[name1]]-=a_batting;
        }
    }
    function getMatchResult() public view returns (string memory) {
        if(result==1) {
            return("A wins");
        }
        else if(result==2) {
            return("B wins");
        }
        else {
            return("tie");
        }
    }
    function getBalance() public view returns(uint, uint, uint) {
        return ( balanceOf[addA], balanceOf[addB],balanceOf[address(this)]);
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/Rsp.sol


In [56]:
!solc --gas --abi --bin src/Rsp.sol


======= src/Rsp.sol:Rsp =======
Gas estimation:
construction:
   41426 + 1108200 = 1149626
external:
   deposit(uint256):	41658
   distributeBetAmount():	infinite
   getBalance():	1623
   getMatchResult():	infinite
   play():	infinite
   setA():	infinite
   setAddressAB(address):	40834
   setB(uint256,uint256):	infinite
Binary: 
60806040527fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff60005534801561003457600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506115a5806100856000396000f3fe60806040526004361061007b5760003560e01c806393e84cd91161004e57806393e84cd9146101b1578063a0be28f5146101c8578063b6b55f25146101d2578063f3180546146102005761007b565b806312065fe0146100805780631fc376f7146100b95780634a686381146100d0578063504e31f214610160575b600080fd5b34801561008c57600080fd5b50610095610245565b60405180848152602001838152602001828152602001935050505060405180910390f35b3480156100c557600080fd5b506100

### ganache@8345

In [41]:
%%writefile src/RspDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"}],"name":"setAddressAB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_batting","type":"uint256"},{"name":"_rsp","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="0x" + "60806040527fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff60005534801561003457600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506115a5806100856000396000f3fe60806040526004361061007b5760003560e01c806393e84cd91161004e57806393e84cd9146101b1578063a0be28f5146101c8578063b6b55f25146101d2578063f3180546146102005761007b565b806312065fe0146100805780631fc376f7146100b95780634a686381146100d0578063504e31f214610160575b600080fd5b34801561008c57600080fd5b50610095610245565b60405180848152602001838152602001828152602001935050505060405180910390f35b3480156100c557600080fd5b506100ce610359565b005b3480156100dc57600080fd5b506100e5610531565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561012557808201518184015260208101905061010a565b50505050905090810190601f1680156101525780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b34801561016c57600080fd5b506101af6004803603602081101561018357600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506105ff565b005b3480156101bd57600080fd5b506101c6610684565b005b6101d0610aa6565b005b6101fe600480360360208110156101e857600080fd5b81019080803590602001909291905050506111cf565b005b34801561020c57600080fd5b506102436004803603604081101561022357600080fd5b810190808035906020019092919080359060200190929190505050611316565b005b600080600060076000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205460076000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054600760003073ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054925092509250909192565b60006003424460405160200180838152602001828152602001925050506040516020818303038152906040528051906020012060001c8161039657fe5b0660ff1690506103a46114aa565b60405180606001604052806040518060400160405280600881526020017f506c617965725f4100000000000000000000000000000000000000000000000081525081526020016103e881526020018381525090508060056000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008201518160000190805190602001906104769291906114cb565b506020820151816001015560408201518160020155905050600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600660405180807f506c617965725f410000000000000000000000000000000000000000000000008152506008019050908152602001604051809103902060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055505050565b60606001600054141561057b576040518060400160405280600681526020017f412077696e73000000000000000000000000000000000000000000000000000081525090506105fc565b600260005414156105c3576040518060400160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525090506105fc565b6040518060400160405280600381526020017f746965000000000000000000000000000000000000000000000000000000000081525090505b90565b30600260006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555080600360006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050565b60606040518060400160405280600881526020017f506c617965725f41000000000000000000000000000000000000000000000000815250905060606040518060400160405280600881526020017f506c617965725f4200000000000000000000000000000000000000000000000081525090506107006114aa565b600560006006856040518082805190602001908083835b6020831061073a5780518252602082019150602081019050602083039250610717565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f01602080910402602001604051908101604052809291908181526020018280546001816001161561010002031660029004801561086c5780601f106108415761010080835404028352916020019161086c565b820191906000526020600020905b81548152906001019060200180831161084f57829003601f168201915b505050505081526020016001820154815260200160028201548152505090506108936114aa565b600560006006856040518082805190602001908083835b602083106108cd57805182526020820191506020810190506020830392506108aa565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156109ff5780601f106109d4576101008083540402835291602001916109ff565b820191906000526020600020905b8154815290600101906020018083116109e257829003601f168201915b5050505050815260200160018201548152602001600282015481525050905060008260400151905060008260400151905080821415610a445760008081905550610a9e565b600082148015610a545750600181145b80610a6b5750600182148015610a6a5750600281145b5b80610a825750600282148015610a815750600081145b5b15610a94576001600081905550610a9d565b60026000819055505b5b505050505050565b60606040518060400160405280600881526020017f506c617965725f41000000000000000000000000000000000000000000000000815250905060606040518060400160405280600881526020017f506c617965725f420000000000000000000000000000000000000000000000008152509050610b226114aa565b600560006006856040518082805190602001908083835b60208310610b5c5780518252602082019150602081019050602083039250610b39565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610c8e5780601f10610c6357610100808354040283529160200191610c8e565b820191906000526020600020905b815481529060010190602001808311610c7157829003601f168201915b50505050508152602001600182015481526020016002820154815250509050610cb56114aa565b600560006006856040518082805190602001908083835b60208310610cef5780518252602082019150602081019050602083039250610ccc565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610e215780601f10610df657610100808354040283529160200191610e21565b820191906000526020600020905b815481529060010190602001808311610e0457829003601f168201915b505050505081526020016001820154815260200160028201548152505090506000826020015190506000826020015190506001600054141561100e5780600760006006886040518082805190602001908083835b60208310610e985780518252602082019150602081019050602083039250610e75565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254039250508190555080600760006006896040518082805190602001908083835b60208310610f6e5780518252602082019150602081019050602083039250610f4b565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506111c7565b600260005414156111c65781600760006006886040518082805190602001908083835b602083106110545780518252602082019150602081019050602083039250611031565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555081600760006006896040518082805190602001908083835b6020831061112a5780518252602082019150602081019050602083039250611107565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055505b5b505050505050565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461122957600080fd5b80341461123557600080fd5b8060076000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055508060076000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555050565b61131e6114aa565b60405180606001604052806040518060400160405280600881526020017f506c617965725f4200000000000000000000000000000000000000000000000081525081526020018481526020018381525090508060056000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008201518160000190805190602001906113ee9291906114cb565b506020820151816001015560408201518160020155905050600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600660405180807f506c617965725f420000000000000000000000000000000000000000000000008152506008019050908152602001604051809103902060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550505050565b60405180606001604052806060815260200160008152602001600081525090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061150c57805160ff191683800117855561153a565b8280016001018555821561153a579182015b8281111561153957825182559160200191906001019061151e565b5b509050611547919061154b565b5090565b61156d91905b80821115611569576000816000905550600101611551565b5090565b9056fea265627a7a72305820bd82e270f3644df05212658c69d05fdee303480a02ccab9c2e5d892c9f2aaaef64736f6c634300050a0032";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 2000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/RspDeploy.js


In [42]:
!node src/RspDeploy.js

Deploying the contract from 0x6Af8a26CC4B8399D35dCF630E4f8D6D84aB56b39
hash: 0x26f3991ab842d071636b8d1d5169689acf6adc330dd1d8c3bd280e451673ea5c
---> The contract deployed to: 0x9f38CC5732781175B1125b0dEF1e286241107806


In [43]:
!geth --exec "eth.getTransactionReceipt('0x26f3991ab842d071636b8d1d5169689acf6adc330dd1d8c3bd280e451673ea5c')" attach http://localhost:8345

{
  blockHash: "0x9a6f1fc61c2d253e986be266db6f8a8c6ae33a97c99f523f461f9d7ce6ca03f2",
  blockNumber: 1,
  contractAddress: "0x9f38cc5732781175b1125b0def1e286241107806",
  cumulativeGasUsed: 1288538,
  from: "0x6af8a26cc4b8399d35dcf630e4f8d6d84ab56b39",
  gasUsed: 1288538,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0x26f3991ab842d071636b8d1d5169689acf6adc330dd1d8c3bd280e451673ea5c",
  transactionIndex: 0
}


In [44]:
%%writefile src/RspUse.js
var Web3=require('web3');
var fs=require('fs');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"}],"name":"setAddressAB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_batting","type":"uint256"},{"name":"_rsp","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];

async function doIt() {
    var rsp= new web3.eth.Contract(_abiArray, '0x9f38CC5732781175B1125b0dEF1e286241107806');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    
    await rsp.methods.setAddressAB(accounts[1]).send({from: accounts[0], gas: 122209});
    await rsp.methods.deposit(10000).send({from: accounts[0], gas: 122209,value:10000});
    rsp.methods.getBalance().call().then(function(value) {console.log("<Before game>\nA:", value[0], "  B:", value[1], "  Contract:",value[2],"\n")});
    await rsp.methods.setA().send({from: accounts[0], gas: 122209});
    await rsp.methods.setB(1000,2).send({from: accounts[0], gas: 122209});
    await rsp.methods.play().send({from: accounts[0], gas: 122209});
    rsp.methods.getMatchResult().call().then(console.log);
    await rsp.methods.distributeBetAmount().send({from: accounts[0], gas: 122209});
    rsp.methods.getBalance().call().then(function(value) {console.log("\n<After game>\nA:", value[0], "  B:", value[1], "  Contract:",value[2])});
}
doIt();

Overwriting src/RspUse.js


### A가 이긴 경우

In [49]:
!node src/RspUse.js

Account: 0x6Af8a26CC4B8399D35dCF630E4f8D6D84aB56b39
<Before game>
A: 47000   B: 53000   Contract: 47000 

A wins

<After game>
A: 48000   B: 52000   Contract: 48000


In [50]:
!node src/RspUse.js

Account: 0x6Af8a26CC4B8399D35dCF630E4f8D6D84aB56b39
<Before game>
A: 58000   B: 62000   Contract: 58000 

A wins

<After game>
A: 59000   B: 61000   Contract: 59000


### B가 이긴 경우

In [51]:
!node src/RspUse.js

Account: 0x6Af8a26CC4B8399D35dCF630E4f8D6D84aB56b39
<Before game>
A: 69000   B: 71000   Contract: 69000 

B wins

<After game>
A: 68000   B: 72000   Contract: 68000


In [52]:
!node src/RspUse.js

Account: 0x6Af8a26CC4B8399D35dCF630E4f8D6D84aB56b39
<Before game>
A: 78000   B: 82000   Contract: 78000 

B wins

<After game>
A: 77000   B: 83000   Contract: 77000


### 비긴 경우

In [53]:
!node src/RspUse.js

Account: 0x6Af8a26CC4B8399D35dCF630E4f8D6D84aB56b39
<Before game>
A: 87000   B: 93000   Contract: 87000 

tie

<After game>
A: 87000   B: 93000   Contract: 87000


### geth@8445

In [1]:
%%writefile src/RspDeploy_geth.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"}],"name":"setAddressAB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_batting","type":"uint256"},{"name":"_rsp","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="0x" + "60806040527fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff60005534801561003457600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506115a5806100856000396000f3fe60806040526004361061007b5760003560e01c806393e84cd91161004e57806393e84cd9146101b1578063a0be28f5146101c8578063b6b55f25146101d2578063f3180546146102005761007b565b806312065fe0146100805780631fc376f7146100b95780634a686381146100d0578063504e31f214610160575b600080fd5b34801561008c57600080fd5b50610095610245565b60405180848152602001838152602001828152602001935050505060405180910390f35b3480156100c557600080fd5b506100ce610359565b005b3480156100dc57600080fd5b506100e5610531565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561012557808201518184015260208101905061010a565b50505050905090810190601f1680156101525780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b34801561016c57600080fd5b506101af6004803603602081101561018357600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506105ff565b005b3480156101bd57600080fd5b506101c6610684565b005b6101d0610aa6565b005b6101fe600480360360208110156101e857600080fd5b81019080803590602001909291905050506111cf565b005b34801561020c57600080fd5b506102436004803603604081101561022357600080fd5b810190808035906020019092919080359060200190929190505050611316565b005b600080600060076000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205460076000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054600760003073ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054925092509250909192565b60006003424460405160200180838152602001828152602001925050506040516020818303038152906040528051906020012060001c8161039657fe5b0660ff1690506103a46114aa565b60405180606001604052806040518060400160405280600881526020017f506c617965725f4100000000000000000000000000000000000000000000000081525081526020016103e881526020018381525090508060056000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008201518160000190805190602001906104769291906114cb565b506020820151816001015560408201518160020155905050600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600660405180807f506c617965725f410000000000000000000000000000000000000000000000008152506008019050908152602001604051809103902060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055505050565b60606001600054141561057b576040518060400160405280600681526020017f412077696e73000000000000000000000000000000000000000000000000000081525090506105fc565b600260005414156105c3576040518060400160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525090506105fc565b6040518060400160405280600381526020017f746965000000000000000000000000000000000000000000000000000000000081525090505b90565b30600260006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555080600360006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050565b60606040518060400160405280600881526020017f506c617965725f41000000000000000000000000000000000000000000000000815250905060606040518060400160405280600881526020017f506c617965725f4200000000000000000000000000000000000000000000000081525090506107006114aa565b600560006006856040518082805190602001908083835b6020831061073a5780518252602082019150602081019050602083039250610717565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f01602080910402602001604051908101604052809291908181526020018280546001816001161561010002031660029004801561086c5780601f106108415761010080835404028352916020019161086c565b820191906000526020600020905b81548152906001019060200180831161084f57829003601f168201915b505050505081526020016001820154815260200160028201548152505090506108936114aa565b600560006006856040518082805190602001908083835b602083106108cd57805182526020820191506020810190506020830392506108aa565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156109ff5780601f106109d4576101008083540402835291602001916109ff565b820191906000526020600020905b8154815290600101906020018083116109e257829003601f168201915b5050505050815260200160018201548152602001600282015481525050905060008260400151905060008260400151905080821415610a445760008081905550610a9e565b600082148015610a545750600181145b80610a6b5750600182148015610a6a5750600281145b5b80610a825750600282148015610a815750600081145b5b15610a94576001600081905550610a9d565b60026000819055505b5b505050505050565b60606040518060400160405280600881526020017f506c617965725f41000000000000000000000000000000000000000000000000815250905060606040518060400160405280600881526020017f506c617965725f420000000000000000000000000000000000000000000000008152509050610b226114aa565b600560006006856040518082805190602001908083835b60208310610b5c5780518252602082019150602081019050602083039250610b39565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610c8e5780601f10610c6357610100808354040283529160200191610c8e565b820191906000526020600020905b815481529060010190602001808311610c7157829003601f168201915b50505050508152602001600182015481526020016002820154815250509050610cb56114aa565b600560006006856040518082805190602001908083835b60208310610cef5780518252602082019150602081019050602083039250610ccc565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020604051806060016040529081600082018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610e215780601f10610df657610100808354040283529160200191610e21565b820191906000526020600020905b815481529060010190602001808311610e0457829003601f168201915b505050505081526020016001820154815260200160028201548152505090506000826020015190506000826020015190506001600054141561100e5780600760006006886040518082805190602001908083835b60208310610e985780518252602082019150602081019050602083039250610e75565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254039250508190555080600760006006896040518082805190602001908083835b60208310610f6e5780518252602082019150602081019050602083039250610f4b565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506111c7565b600260005414156111c65781600760006006886040518082805190602001908083835b602083106110545780518252602082019150602081019050602083039250611031565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555081600760006006896040518082805190602001908083835b6020831061112a5780518252602082019150602081019050602083039250611107565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055505b5b505050505050565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461122957600080fd5b80341461123557600080fd5b8060076000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055508060076000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555050565b61131e6114aa565b60405180606001604052806040518060400160405280600881526020017f506c617965725f4200000000000000000000000000000000000000000000000081525081526020018481526020018381525090508060056000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008201518160000190805190602001906113ee9291906114cb565b506020820151816001015560408201518160020155905050600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600660405180807f506c617965725f420000000000000000000000000000000000000000000000008152506008019050908152602001604051809103902060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550505050565b60405180606001604052806060815260200160008152602001600081525090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061150c57805160ff191683800117855561153a565b8280016001018555821561153a579182015b8281111561153957825182559160200191906001019061151e565b5b509050611547919061154b565b5090565b61156d91905b80821115611569576000816000905550600101611551565b5090565b9056fea265627a7a72305820bd82e270f3644df05212658c69d05fdee303480a02ccab9c2e5d892c9f2aaaef64736f6c634300050a0032";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 2000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/RspDeploy_geth.js


In [30]:
!node src/RspDeploy_geth.js

Deploying the contract from 0xB2229d3b848C2A21990C230DFED7f87DFF04b12d
hash: 0xe31c58bcc4f1d84297b1b90f76b4a48fd263d2a5cb9dc4362217288f79a0fb5c
---> The contract deployed to: 0xBbcf2Fcd50edB7306F34E24Ce9968E0FFaa9076E


In [31]:
!geth --exec "eth.getTransactionReceipt('0xe31c58bcc4f1d84297b1b90f76b4a48fd263d2a5cb9dc4362217288f79a0fb5c')" attach http://localhost:8445

{
  blockHash: "0xebb28afae73adde43e8ba2dfd520aae58036732e444df34246ff75001bbf2a76",
  blockNumber: 118,
  contractAddress: "0xbbcf2fcd50edb7306f34e24ce9968e0ffaa9076e",
  cumulativeGasUsed: 1288538,
  from: "0xb2229d3b848c2a21990c230dfed7f87dff04b12d",
  gasUsed: 1288538,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0xe31c58bcc4f1d84297b1b90f76b4a48fd263d2a5cb9dc4362217288f79a0fb5c",
  transactionIndex: 0
}


In [32]:
%%writefile src/RspUse_geth.js
var Web3=require('web3');
var fs=require('fs');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"}],"name":"setAddressAB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_batting","type":"uint256"},{"name":"_rsp","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];

async function doIt() {
    var rsp= new web3.eth.Contract(_abiArray, '0xBbcf2Fcd50edB7306F34E24Ce9968E0FFaa9076E');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    
    await rsp.methods.setAddressAB(accounts[1]).send({from: accounts[0], gas: 122209});
    await rsp.methods.deposit(10000).send({from: accounts[0], gas: 122209,value:10000});
    rsp.methods.getBalance().call().then(function(value) {console.log("<Before game>\nA:", value[0], "  B:", value[1], "  Contract:",value[2],"\n")});
    await rsp.methods.setA().send({from: accounts[0], gas: 122209});
    await rsp.methods.setB(1000,2).send({from: accounts[0], gas: 122209});
    await rsp.methods.play().send({from: accounts[0], gas: 122209});
    rsp.methods.getMatchResult().call().then(console.log);
    await rsp.methods.distributeBetAmount().send({from: accounts[0], gas: 122209});
    rsp.methods.getBalance().call().then(function(value) {console.log("\n<After game>\nA:", value[0], "  B:", value[1], "  Contract:",value[2])});
}
doIt();

Overwriting src/RspUse_geth.js


### A가 이긴 경우

In [45]:
!node src/RspUse_geth.js

Account: 0xB2229d3b848C2A21990C230DFED7f87DFF04b12d
<Before game>
A: 131000   B: 129000   Contract: 131000 

A wins

<After game>
A: 132000   B: 128000   Contract: 132000


### 비긴 경우

In [46]:
!node src/RspUse_geth.js

Account: 0xB2229d3b848C2A21990C230DFED7f87DFF04b12d
<Before game>
A: 142000   B: 138000   Contract: 142000 

tie

<After game>
A: 142000   B: 138000   Contract: 142000


### B가 이긴 경우

In [47]:
!node src/RspUse_geth.js

Account: 0xB2229d3b848C2A21990C230DFED7f87DFF04b12d
<Before game>
A: 152000   B: 148000   Contract: 152000 

B wins

<After game>
A: 151000   B: 149000   Contract: 151000
